# Loading the dataset

In [ ]:
import pandas as pd

df = pd.read_csv("/train_aWnotuB.csv")
df

In [ ]:
import numpy as np
df.info()

# EDA

In [ ]:
df.isnull().sum()

In [ ]:
df.duplicated()

In [ ]:
df = df.drop_duplicates()
df

In [ ]:
df.shape

In [ ]:
df.describe()

In [ ]:
df['DateTime'] = pd.to_datetime(df['DateTime'])  

df['Date'] = df['DateTime'].dt.date
df['Time'] = df['DateTime'].dt.time

df['Date'] = df['Date'].astype(str)
df['Time'] = df['Time'].astype(str)

In [ ]:
df = df.drop('DateTime', axis = 1)

In [ ]:
df

In [ ]:
df['Date'] = pd.to_datetime(df['Date'])

df['DayOfWeek'] = df['Date'].dt.dayofweek

df['Month'] = df['Date'].dt.month

df['Year'] = df['Date'].dt.year

In [ ]:
df.head()

In [ ]:
df.tail()

In [ ]:
import holidays
df['Date'] = pd.to_datetime(df['Date'])

indian_holidays = holidays.India(years=df['Date'].dt.year.unique())

# Custom function to check if the date is a holiday in India
def is_holiday_in_india(date):
    return date in indian_holidays

# Apply the custom function to the 'Date' column
df['IsHoliday'] = df['Date'].apply(lambda x: is_holiday_in_india(x))

In [ ]:
df.head()

In [ ]:
df['IsHoliday'].unique()

In [ ]:
# Removing unnecessary columns

df = df.drop(['ID'], axis = 1)
df.head()

In [ ]:
# Visualization 

import plotly.express as px
import plotly.graph_objects as go
import plotly.io as pio
pio.templates.default = "plotly_white"

In [ ]:
fig = px.line(df, x='Date', y='Vehicles', title='Traffic Volume over Time')
fig.show()

In [ ]:
# Grouping by Junction and looking at the statistical measures
junction_stats = df.groupby('Junction')['Vehicles'].describe()
print(junction_stats)

# Visualization

## Bar plot for junction

In [ ]:
import matplotlib.pyplot as plt
df.groupby('Junction')['Vehicles'].mean().plot(kind='bar')
plt.xlabel('Junction')
plt.ylabel('Average Vehicles')
plt.title('Average Vehicles by Junction')
plt.show()

# Box plot for IsHoliday

In [ ]:
df.boxplot(column='Vehicles', by='IsHoliday')
plt.xlabel('Is Holiday')
plt.ylabel('Vehicles')
plt.title('Distribution of Vehicles by Holiday')
plt.show()

## Hypothesis Testing : t-test

In [ ]:
from scipy.stats import ttest_ind

junction_1 = df[df['Junction'] == 1]['Vehicles']
junction_2 = df[df['Junction'] == 2]['Vehicles']

t_stat, p_value = ttest_ind(junction_1, junction_2)
print(f"t-statistic: {t_stat}, p-value: {p_value}")

### This signifies that the t-statistic of 153.93 indicates a substantial difference between the average number of vehicles for the two junctions.

### A p-value of 0.0 indicates an extremely low probability of observing the data under the assumption that there is no difference between the means of the two groups, suggesting that the null hypothesis can be rejected.

# Feature Engineering

## Creating lag features to capture any time-dependant features

In [ ]:
df['PreviousDayVehicles'] = df['Vehicles'].shift(24)
df['PreviousHourVehicles'] = df['Vehicles'].shift(1)

In [ ]:
df.head()

In [ ]:
df.isnull().sum()

In [ ]:
df['PreviousDayVehicles'].fillna(0,inplace = True)
df['PreviousHourVehicles'].fillna(0,inplace = True)
df.head()

# Splitting the Data:


In [ ]:
from sklearn.model_selection import train_test_split

train_df, test_df = train_test_split(df, test_size=0.2, shuffle=False)

# Printing the sizes of the training and testing sets
print(f"Training set size: {len(train_df)}")
print(f"Testing set size: {len(test_df)}")

# Modeling and Evaluation

In [ ]:
plt.figure(figsize=(12, 6))
plt.plot(target.index, target.values, label='Historical Traffic', color='b')
plt.plot(train_forecast_df.index, train_forecast_df['Vehicles'], label='Forecasted Traffic', color='r')
plt.xlabel('Time')
plt.ylabel('Traffic (Vehicles)')
plt.title('Historical and Forecasted Traffic Patterns for the Training Set')
plt.legend()
plt.show()

# Transitioning to LSTM

In [ ]:
from keras.models import Sequential
from keras.layers import LSTM, Dense

In [ ]:
df.head()

In [ ]:
from sklearn.preprocessing import MinMaxScaler

# Converting 'Vehicles' column to float
df['Vehicles'] = df['Vehicles'].astype(float)

# Normalizing 'Vehicles' column using MinMaxScaler
scaler = MinMaxScaler()
df[['Vehicles']] = scaler.fit_transform(df[['Vehicles']])

In [ ]:
df.head()

In [ ]:
df.dtypes

In [ ]:
# Convert the 'Date' column to numeric features
df['Year'] = df['Date'].dt.year
df['Month'] = df['Date'].dt.month
df['Day'] = df['Date'].dt.day
df['Hour'] = df['Date'].dt.hour

# Drop the original 'Date' column since it's no longer needed
df.drop(columns=['Date'], inplace=True)

# Check if all columns are now numeric
print(df.dtypes)

In [ ]:
# Convert 'IsHoliday' column to numeric format
df['IsHoliday'] = df['IsHoliday'].astype(int)

In [ ]:
df.head()

In [ ]:
import numpy as np
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense

sequence_length = 24 

X_columns = ['Junction', 'Vehicles', 'DayOfWeek', 'Month', 'Year', 'IsHoliday',
             'PreviousDayVehicles', 'PreviousHourVehicles']
y_column = 'Vehicles'

# Splitting the data into input sequences (X) and labels (y)
X = []
y = []

data_array = df.values

data_array = df[X_columns + [y_column]].values

for i in range(len(data_array) - sequence_length):
    X.append(data_array[i:i + sequence_length, :-1])  # Exclude last column
    y.append(data_array[i + sequence_length, -1])  # Last column is target 'Vehicles'

X = np.array(X)
y = np.array(y)

# Splitting the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:
# Building the LSTM model
model = Sequential()
model.add(LSTM(units=50, return_sequences=True, input_shape=(X_train.shape[1], X_train.shape[2])))
model.add(LSTM(units=50))
model.add(Dense(units=1))

# Compiling the model
model.compile(optimizer='adam', loss='mean_squared_error')

# Training the model on the training data
model.fit(X_train, y_train, epochs=30, batch_size=32, validation_split=0.2)

In [ ]:
# Evaluation 

loss = model.evaluate(X_test, y_test)
print(f"Test Loss: {loss}")

# Compare with training loss
train_loss = model.evaluate(X_train, y_train)
print(f"Training Loss: {train_loss}")

In [ ]:
y_pred = model.predict(X_test)

# Reshaping y_pred to a 2D array with shape (num_samples, 1)
y_pred = y_pred.reshape(-1, 1)
y_test = y_test.reshape(-1, 1)


# Inverse transforming the predicted values to their original scale (if needed)
y_pred_original = scaler.inverse_transform(y_pred)

# Inverse transforming the actual test target values (y_test) to their original scale
y_test_original = scaler.inverse_transform(y_test)

In [ ]:
# Visualizing the results: 

# Plotting the actual 'Vehicles' values against the predicted values for the test dataset
plt.figure(figsize=(12, 6))
plt.plot(y_test_original, label='Actual Vehicles')
plt.plot(y_pred_original, label='Predicted Vehicles')
plt.xlabel('Time')
plt.ylabel('Vehicles')
plt.title('Actual vs. Predicted Vehicles')
plt.legend()
plt.show()

In [ ]:
from sklearn.metrics import mean_absolute_error, mean_squared_error

mae = mean_absolute_error(y_test_original, y_pred_original)
mse = mean_squared_error(y_test_original, y_pred_original)
rmse = np.sqrt(mse)

print("Mean Absolute Error (MAE):", mae)
print("Mean Squared Error (MSE):", mse)
print("Root Mean Squared Error (RMSE):", rmse)

In conclusion, the LSTM model demonstrates strong predictive capabilities for traffic forecasting. Its accurate predictions and low error metrics make it a valuable tool for traffic management and infrastructure planning in smart cities. However, it's essential to continue monitoring and refining the model regularly to ensure it maintains its accuracy over time and adapts to any changes in traffic patterns or other factors affecting traffic.

# Analyzing Forecasted Traffic Patterns

## Plotting the Forecasted vs. Actual Traffic Counts

In [ ]:
df['DateTime'] = pd.to_datetime(df[['Year', 'Month', 'Day', 'Hour']])
df.head()

In [ ]:
# Getting the timestamp for the test data
test_timestamps = df.iloc[-len(y_test_original):]['DateTime']

# Creating a figure and plot the actual traffic counts
plt.figure(figsize=(12, 6))
plt.plot(test_timestamps, y_test_original, label='Actual Traffic Counts', color='blue')

# Getting the timestamp for the forecasted data
forecasted_timestamps = df.iloc[-len(y_test_original):]['DateTime']

# Plotting the predicted traffic counts for the corresponding time period
plt.plot(forecasted_timestamps, y_pred_original, label='Forecasted Traffic Counts', color='red')

plt.xlabel('Date')
plt.ylabel('Traffic Counts')
plt.title('Forecasted vs. Actual Traffic Counts')
plt.legend()
plt.grid(True)
plt.show()

Since the predicted traffic counts only occupy a small part at the end of the graph, it indicates that the model's predictions are trailing the actual traffic counts for the test dataset.

## Visualizing Trends

This plot will help us identify any monthly trends in traffic.

In [ ]:
import seaborn as sns
# Extracting Year and Month from the DateTime column
df['Year'] = df['DateTime'].dt.year
df['Month'] = df['DateTime'].dt.month

# Grouping the data by Year and Month and calculate the average traffic counts for each month
monthly_traffic = df.groupby(['Year', 'Month'])['Vehicles'].mean().reset_index()

# Plotting the average traffic counts by month
plt.figure(figsize=(10, 6))
sns.lineplot(x='Month', y='Vehicles', hue='Year', data=monthly_traffic)
plt.xlabel('Month')
plt.ylabel('Average Traffic Counts')
plt.title('Monthly Average Traffic Counts')
plt.legend(title='Year', loc='upper right')
plt.grid(True)
plt.show()

## Analyzing Seasonal Variations

This plot with the seasonal decomposition will reveal the seasonal variations and recurring patterns in the forecasted data.

In [ ]:
import statsmodels.api as sm

# Decomposing the time series into trend, seasonal, and residual components
decomposition = sm.tsa.seasonal_decompose(df['Vehicles'], model='additive', period=24*7)  # Assuming a weekly seasonality

# Plotting the components
plt.figure(figsize=(12, 8))
plt.subplot(4, 1, 1)
plt.plot(decomposition.trend)
plt.ylabel('Trend')
plt.title('Trend Component')
plt.grid(True)

plt.subplot(4, 1, 2)
plt.plot(decomposition.seasonal)
plt.ylabel('Seasonal')
plt.title('Seasonal Component')
plt.grid(True)

plt.subplot(4, 1, 3)
plt.plot(decomposition.resid)
plt.ylabel('Residual')
plt.title('Residual Component')
plt.grid(True)

plt.subplot(4, 1, 4)
plt.plot(df['Vehicles'])
plt.ylabel('Traffic Counts')
plt.title('Original Data')
plt.grid(True)

plt.tight_layout()
plt.show()


# Infrastructure Planning

## Design a Robust Traffic Management System

Designing a dynamic traffic management system that adapts to peak traffic hours and efficiently handles the increased load during those hours.

In [ ]:
df['HourOfDay'] = df['DateTime'].dt.hour
df.head()

In [ ]:
peak_traffic_hours = df[(df['HourOfDay'] >= 7) & (df['HourOfDay'] <= 10)]  # Assumed peak traffic hours from 7 AM to 10 AM

# Visualize peak traffic hours
plt.figure(figsize=(12, 6))
plt.plot(df['DateTime'], df['Vehicles'], label='Traffic Counts')
plt.scatter(peak_traffic_hours['DateTime'], peak_traffic_hours['Vehicles'], color='purple', label='Peak Traffic Hours')
plt.xlabel('Date')
plt.ylabel('Traffic Counts')
plt.title('Peak Traffic Hours')
plt.legend()
plt.grid(True)
plt.show()